In [2]:
# !apt-get install -y fluidsynth
!pip install midiutil
!pip install midi2audio
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
import base64
import os

import requests
from midiutil import MIDIFile
from tqdm.notebook import tqdm

In [12]:
# Paths
midi_dir = "jazz-midi"
os.makedirs(midi_dir, exist_ok=True)

soundfont_path = "FluidR3_GM.sf2"

output_dir = "jazz-audio"
os.makedirs(output_dir, exist_ok=True)

In [25]:
# NOTE: the open voicings are not just [x if x > k else x+12 for x in closed]
# because the 9 can either be in the same octave or an octave up. Similarly,
# some closed chords have the 3 and others have the 15. I still think I could
# have done something smarter here, but whatever
voicings = {
    "Maj7_closed": [0, 4, 11],
    "7_closed": [0, 4, 10],
    "m7_closed": [0, 3, 10],
    "Maj6_closed": [0, 4, 9, 11],
    "69_closed": [0, 4, 9, 14],
    "Maj9_closed": [0, 4, 11, 14],
    "9_closed": [0, 4, 10, 14],
    "m9_closed": [0, 3, 10, 14],
    "7b9_closed": [0, 4, 10, 13],
    "7#9_closed": [0, 4, 10, 15],
    "7#11_closed": [0, 4, 6, 10],
    "7b13_closed": [0, 4, 8, 10],
    "7add13_closed": None,  # [0, 4, 9, 10], (rarely voiced this way)
    "13_closed": [0, 4, 9, 10, 14],
    "7alt_closed": [0, 4, 8, 10, 15],
    "Maj7_open": [0, 11, 16],
    "7_open": [0, 10, 16],
    "m7_open": [0, 10, 15],
    "Maj6_open": [0, 11, 16, 21],
    "69_open": [0, 9, 14, 16],
    "Maj9_open": [0, 11, 14, 16],
    "9_open": [0, 10, 14, 16],
    "m9_open": [0, 10, 14, 15],
    "7b9_open": [0, 10, 13, 16],
    "7#9_open": None,  # [0, 10, 15, 16] (rarely voiced this way)
    "7#11_open": [0, 10, 16, 18],
    "7b13_open": [0, 10, 16, 20],
    "7add13_open": [0, 10, 16, 21],
    "13_open": [0, 10, 14, 16, 21],
    "7alt_open": None,  # [0, 10, 15, 16, 20] (rarely voiced this way)
    "dim_closed": [0, 3, 6, 9],
    "dim_clopen": [0, 6, 9, 15],
    "dim_open": [0, 9, 15, 18],
    "hdim_closed": [0, 3, 6, 10],
    "hdim_clopen": [0, 6, 10, 15],
    "hdim_open": [0, 10, 15, 18],  # same as m7b13
}

for voicing, offsets in voicings.items():
    if offsets is not None and voicing.endswith("closed"):
        open_offsets = voicings[voicing.replace("closed", "open")]
        if open_offsets is not None:
            for x in open_offsets:
                assert (
                    x in offsets or x - 12 in offsets
                ), f"Open/close voicing disagree for {voicing}, {offsets}"

roots = {
    "C": 0,
    "Db": 1,
    "D": 2,
    "Eb": 3,
    "E": 4,
    "F": 5,
    "Gb": 6,
    "G": 7,
    "Ab": 8,
    "A": -3,
    "Bb": -2,
    "B": -1,
}

chord_voicings = {
    f"{root}{voicing}": [
        48 + root_offset
        # make non-root notes an octave higher for closed voicings
        + (
            voicing_offset + 12
            if (voicing_offset != 0 and "closed" in voicing)
            else voicing_offset
        )
        for voicing_offset in voicing_offsets
    ]
    for voicing, voicing_offsets in voicings.items()
    for root, root_offset in roots.items()
    if voicing_offsets is not None
}


def create_midi_file(name, notes, velocity=128, duration=3, bpm=120):
    midi = MIDIFile(1)
    track = 0
    lead_in_time = 0.25  # add a 0.25 beat (1/2 second at 120bpm) delay
    midi.addTempo(track, time=0, tempo=bpm)
    for note in notes:
        midi.addNote(
            track,
            channel=0,
            pitch=note,
            time=lead_in_time,
            duration=duration,
            volume=velocity,
        )
    filepath = os.path.join(midi_dir, f"{name}.mid")
    with open(filepath, "wb") as f:
        midi.writeFile(f)


for name, notes in tqdm(chord_voicings.items()):
    create_midi_file(name, notes)

  0%|          | 0/396 [00:00<?, ?it/s]

In [26]:
import multiprocessing
from midi2audio import FluidSynth

def midi_to_wav(filename):
    midi_path = os.path.join(midi_dir, filename)
    wav_path = os.path.join(output_dir, filename.replace(".mid", ".wav"))

    # Each process creates its own FluidSynth instance
    fs = FluidSynth(soundfont_path)
    fs.midi_to_audio(midi_path, wav_path)

if __name__ == "__main__":
    files = [filename for filename in os.listdir(midi_dir) if filename.endswith(".mid")]
    with multiprocessing.Pool() as pool:
        pool.map(midi_to_wav, files)


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'jazz-audio/Abdim_open.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'jazz-audio/AMaj7_open.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'jazz-audio/B9_closed.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'jazz-audio/Bb9_open.wav'..
FluidSynth runtime version 2.2.5
Copyri

In [29]:
import os
import base64
import requests
import multiprocessing
from functools import partial

# CONFIG
deck_name = "jazz-v2"
media_folder = "jazz-audio"
anki_connect_url = "http://172.20.160.1:8765"

# Step 1: Create the deck
def create_deck(name):
    requests.post(
        anki_connect_url,
        json={"action": "createDeck", "version": 6, "params": {"deck": name}},
    )

# Step 2: Prepare the note payload (done in parallel)
def prepare_note_payload(filename):
    audio_path = os.path.join(media_folder, filename)
    with open(audio_path, "rb") as f:
        audio_data = f.read()
        encoded_audio = base64.b64encode(audio_data).decode("utf-8")
    return {
        "filename": filename,
        "encoded": encoded_audio,
        "back": os.path.splitext(filename)[0]
    }

# Step 3: Send to Anki (sequentially, safer)
def send_to_anki(note):
    filename = note["filename"]
    encoded_audio = note["encoded"]
    back = note["back"]

    # Upload audio
    requests.post(
        anki_connect_url,
        json={
            "action": "storeMediaFile",
            "version": 6,
            "params": {"filename": filename, "data": encoded_audio},
        },
    )

    # Add note
    requests.post(
        anki_connect_url,
        json={
            "action": "addNote",
            "version": 6,
            "params": {
                "note": {
                    "deckName": deck_name,
                    "modelName": "Basic",
                    "fields": {
                        "Front": f"[sound:{filename}]",
                        "Back": back,
                    },
                    "options": {"allowDuplicate": False},
                    "tags": ["jazz-audio"],
                }
            },
        },
    )
    print(f"Added {filename}")

# MAIN
if __name__ == "__main__":
    create_deck(deck_name)

    # Get relevant files
    files = [
        f for f in os.listdir(media_folder)
        if f.lower().endswith((".mp3", ".wav", ".ogg"))
    ]

    # Prepare notes using multiprocessing
    with multiprocessing.Pool() as pool:
        notes = pool.map(prepare_note_payload, files)

    # Send notes to Anki (sequential to avoid AnkiConnect issues)
    for note in notes:
        send_to_anki(note)

    print("✅ Done!")


Added A13_closed.wav
Added A13_open.wav
Added A69_closed.wav
Added A69_open.wav
Added A7#11_closed.wav
Added A7#11_open.wav
Added A7#9_closed.wav
Added A7add13_open.wav
Added A7alt_closed.wav
Added A7b13_closed.wav
Added A7b13_open.wav
Added A7b9_closed.wav
Added A7b9_open.wav
Added A7_closed.wav
Added A7_open.wav
Added A9_closed.wav
Added A9_open.wav
Added Ab13_closed.wav
Added Ab13_open.wav
Added Ab69_closed.wav
Added Ab69_open.wav
Added Ab7#11_closed.wav
Added Ab7#11_open.wav
Added Ab7#9_closed.wav
Added Ab7add13_open.wav
Added Ab7alt_closed.wav
Added Ab7b13_closed.wav
Added Ab7b13_open.wav
Added Ab7b9_closed.wav
Added Ab7b9_open.wav
Added Ab7_closed.wav
Added Ab7_open.wav
Added Ab9_closed.wav
Added Ab9_open.wav
Added Abdim_clopen.wav
Added Abdim_closed.wav
Added Abdim_open.wav
Added Abhdim_clopen.wav
Added Abhdim_closed.wav
Added Abhdim_open.wav
Added Abm7_closed.wav
Added Abm7_open.wav
Added Abm9_closed.wav
Added Abm9_open.wav
Added AbMaj6_closed.wav
Added AbMaj6_open.wav
Added Ab